In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [2]:
import yaml
from easydict import EasyDict as edict

In [3]:
config_file = glob('./config/GTS/*.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [4]:
def plot_lambda_raster(im_o, im_t):
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(90, 80))
    time = im_o.shape[0] / 10000
    extent = [0, time, 0, 100]
    im_o = axes[0].imshow(np.transpose(im_o), cmap='binary', extent=extent)
    im_t = axes[1].imshow(np.transpose(im_t), cmap='binary', extent=extent)
    ylabel = 'Lambda'
    axes[0].set_title('Prediction\n', fontsize = 30)
    axes[0].set_xlabel('Time(s)', fontsize = 30)
    axes[0].set_ylabel('Node index', fontsize = 30)
    axes[0].set_aspect(0.1)
    axes[0].tick_params(axis='x', labelsize=20)
    axes[0].tick_params(axis='y', labelsize=20)
    axes[1].set_title('Target\n', fontsize = 30)
    axes[1].set_xlabel('Time(s)', fontsize = 30)
    axes[1].set_ylabel('Node index', fontsize = 30)
    axes[1].set_aspect(0.1)
    axes[1].tick_params(axis='x', labelsize=20)
    axes[1].tick_params(axis='y', labelsize=20)
    cax_o = fig.add_axes([axes[0].get_position().x1+0.01,axes[0].get_position().y0,0.02,axes[0].get_position().height])
    cax_t = fig.add_axes([axes[1].get_position().x1+0.01,axes[1].get_position().y0,0.02,axes[1].get_position().height])
    cbar1 = plt.colorbar(im_o, cax=cax_o)
    cbar2 = plt.colorbar(im_t, cax=cax_t)
    cbar1.ax.set_ylabel(ylabel, fontsize=30)
    cbar1.ax.tick_params(axis='y', labelsize=20)
    cbar2.ax.set_ylabel(ylabel, fontsize=30)
    cbar2.ax.tick_params(axis='y', labelsize=20)
#     plt.savefig(‘./fig/lamall_RNN_{}.png’.format(suffix), bbox_inches=‘tight’)


In [3]:
from utils.utils import build_fully_connected_edge_idx

In [4]:
import pickle

import torch
from tqdm import tqdm

In [8]:
spike = pickle.load(open('./data/LNP_spk_all.pickle', 'rb'))
lam = pickle.load(open('./data/LNP_lam_all.pickle', 'rb'))

In [5]:
from torch_geometric.data import Data, Batch, Dataset, InMemoryDataset
from torch_geometric.loader import DataLoader

In [6]:
from dataset.torch_geometric_spike_dataset import *

In [11]:
config.dataset.train_step

4000000

In [12]:
data = spike[:, :config.dataset.train_step]

In [13]:
data.shape

(100, 4000000)

In [14]:
train_dataset = Train_Dataset(root=config.dataset.root)

Processing...
Done!


In [15]:
train_dataset

Train_Dataset(39999)

In [111]:
train_loader = DataLoader(train_dataset, batch_size=128)

In [112]:
next(iter(train_loader))

DataBatch(x=[12800, 200], y=[12800, 20, 2], batch=[12800], ptr=[129])

In [121]:
for data_batch in train_loader:
    print(data_batch.x.shape[0]//100)

128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128


In [128]:
use_gpu = True
device = 'gpu'

In [130]:
if (use_gpu) and (device != 'cpu'):
    print("O")

O


In [123]:
data_batch.to('cpu')

DataBatch(x=[6300, 200], y=[6300, 20, 2], batch=[6300], ptr=[64])

In [117]:
data_batch.batch.shape

torch.Size([6300])

In [119]:
data_batch.x.shape[0] / 

6300

In [98]:
data_batch.x.shape

torch.Size([300, 200])

In [99]:
data_batch.y.shape

torch.Size([300, 20, 2])

In [103]:
data_batch.y[:100,:,0]

tensor([[0.0157, 0.0159, 0.0160,  ..., 0.0165, 0.0167, 0.0168],
        [0.0132, 0.0134, 0.0136,  ..., 0.0141, 0.0143, 0.0144],
        [0.0097, 0.0100, 0.0102,  ..., 0.0107, 0.0109, 0.0111],
        ...,
        [0.0165, 0.0166, 0.0167,  ..., 0.0166, 0.0167, 0.0169],
        [0.0174, 0.0175, 0.0177,  ..., 0.0178, 0.0179, 0.0181],
        [0.0172, 0.0173, 0.0174,  ..., 0.0178, 0.0179, 0.0181]])

In [104]:
data_batch.y[:100,:,1]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [100]:
data_batch.edge_index = new_edge

In [101]:
data_batch.edge_index

tensor([[  0,   0,   0,  ..., 299, 299, 299],
        [  1,   2,   3,  ..., 296, 297, 298]])

In [102]:
data_batch.edge_index.shape

torch.Size([2, 29700])

In [106]:
def build_batch_edge_index(edge_index, num_graphs):
    new_edge = edge_index
    for num in range(1,num_graphs):
        next_graph_edge = edge_index + num*100
        new_edge = torch.cat([new_edge, next_graph_edge], dim=-1)
    
        
    return new_edge

In [107]:
edge_index = build_fully_connected_edge_idx(num_nodes=100)

In [108]:
new_edge = build_batch_edge_index(edge_index, 128)

In [109]:
new_edge.shape

torch.Size([2, 1267200])

In [110]:
new_edge

tensor([[    0,     0,     0,  ..., 12799, 12799, 12799],
        [    1,     2,     3,  ..., 12796, 12797, 12798]])

In [82]:
new_edge[:,:9900]

tensor([[ 0,  0,  0,  ..., 99, 99, 99],
        [ 1,  2,  3,  ..., 96, 97, 98]])

In [83]:
new_edge[:,9900:9900*2]

tensor([[100, 100, 100,  ..., 199, 199, 199],
        [101, 102, 103,  ..., 196, 197, 198]])

In [84]:
new_edge[:,9900*2:9900*3]

tensor([[200, 200, 200,  ..., 299, 299, 299],
        [201, 202, 203,  ..., 296, 297, 298]])

In [86]:
for num in range(1,3):
    print(num)

1
2


In [87]:
9900*3

29700

In [11]:
dirs = glob('./data/spike_lambda/processed/train_n100.pt')[0]

In [12]:
dirs

'./data/spike_lambda/processed/train_n100.pt'

In [13]:
train = torch.load(dirs)

In [15]:
train

(Data(x=[81600000, 20], y=[81600000, 2, 2]),
 defaultdict(dict,
             {'x': tensor([       0,  4800000,  9600000, 14400000, 19200000, 24000000, 28800000,
                      33600000, 38400000, 43200000, 48000000, 52800000, 57600000, 62400000,
                      67200000, 72000000, 76800000, 81600000]),
              'y': tensor([       0,  4800000,  9600000, 14400000, 19200000, 24000000, 28800000,
                      33600000, 38400000, 43200000, 48000000, 52800000, 57600000, 62400000,
                      67200000, 72000000, 76800000, 81600000])}))